<a href="https://colab.research.google.com/github/CheayeonLee/ESAA_OB/blob/main/4%EC%A3%BC%EC%B0%A8%EA%B3%BC%EC%A0%9C(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH9. 비지도 학습

## 9.1 군집

군집: 비슷한 샘플을 구별해 하나의 클러스터 또는 비슷한 샘플의 그룹으로 할당하는 작업
- 로지스틱 회귀, SVM, 랜덤 포레스트 같은 분류 알고리즘이 잘 맞음

### 9.1.1 k-평균
- 반복 몇 번으로 레이블이 없는 데이터셋을 빠르고 효율적으로 클러스터로 묶을 수 있는 간단한 알고리즘
- =로이드-포지 알고리즘

In [1]:
#데이터셋 생성
from sklearn.datasets import make_blobs
import numpy as np


blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [2]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [3]:
y_pred

array([3, 0, 1, ..., 4, 1, 0], dtype=int32)

In [4]:
y_pred is kmeans.labels_ # 분류와 달리 인덱스임

True

In [5]:
kmeans.cluster_centers_  # 센트로이드

array([[-2.80389616,  1.80117999],
       [ 0.20876306,  2.25551336],
       [-1.46679593,  2.28585348],
       [-2.80037642,  1.30082566],
       [-2.79290307,  2.79641063]])

In [6]:
# 새로운 샘플에 가장 가까운 센트로이드의 클러스터를 할당
X_new = np.array([[0,2],[3,2],[-3,3],[-3,2.5]])
kmeans.predict(X_new)

array([1, 1, 4, 4], dtype=int32)

- 클러스터의 크기가 많이 다르면 잘 작동하지 않음(거리를 고려하는 것이 전부이기 때문)

In [7]:
# 샘플과 각 센트로이드 사이의 거리를 반환
kmeans.transform(X_new)

array([[2.81093633, 0.32995317, 1.49439034, 2.88633901, 2.9042344 ],
       [5.80730058, 2.80290755, 4.4759332 , 5.84236351, 5.84739223],
       [1.21475352, 3.29399768, 1.69136631, 1.71086031, 0.29040966],
       [0.72581411, 3.21806371, 1.54808703, 1.21567622, 0.36159148]])

### 센트로이드 초기화 방법

In [8]:
good_init = np.array([[-3,3],[-3,2],[-3,1],[-1,2],[0,2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1)

In [9]:
kmeans.fit(X)
kmeans.score(X)

-211.59853725816836

### k-평균 속도 개선과 미니배치 k-평균

In [10]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


MiniBatchKMeans(n_clusters=5)

### 최적의 클러스터 개수 찾기

In [11]:
# 실루엣 계수: (b-a) / max(a,b)
# a: 동일한 클러스터에 있는 다른 샘플까지 평균 거리
# b: 가장 가까운 클러스터까지 평균 거리
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

0.655517642572828

### 9.1.2 k-평균의 한계

- 알고리즘 여러 번 시행해야 함
- 클러스터 개수 지정해야 함

### 9.1.3 군집을 사용한 이미지 분할

- 이미지를 세그먼트 여러 개로 분할하는 작업
- 시맨틱분할: 동일한 종류의 물체에 속한 모든 픽셀은 같은 세그먼트에 할당

In [12]:
import os
import urllib.request

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "unsupervised_learning"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


images_path = os.path.join(PROJECT_ROOT_DIR, "images", "unsupervised_learning")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
filename = "ladybug.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/unsupervised_learning/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))

('./images/unsupervised_learning/ladybug.png',
 <http.client.HTTPMessage at 0x7de9295d8190>)

In [13]:
from matplotlib.image import imread # or 'from imageio import imread'
image = imread(os.path.join('images','unsupervised_learning','ladybug.png'))
image.shape

(533, 800, 3)

- 3차원: 높이, 너비, 컬러 채널 개수

In [14]:
X = image.reshape(-1,3)
kmeans = KMeans(n_clusters=8).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


### 9.1.4 군집을 사용한 전처리

- 차원 축소에 효과적인 방법
- 지도 학습 알고리즘을 적용하기 전에 전처리 단계로 사용할 수 있음

In [15]:
from sklearn.datasets import load_digits

X_digits, y_digits = load_digits(return_X_y = True)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

In [17]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
log_reg.score(X_test, y_test)

0.9688888888888889

k-평균을 전처리 단계로 사용해 더 좋아지는지 확인

In [19]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('kmeans', KMeans(n_clusters=50)),
    ('log_reg', LogisticRegression()),
])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                ('log_reg', LogisticRegression())])

In [20]:
pipeline.score(X_test, y_test)

0.9622222222222222

In [21]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2,100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Fitting 3 folds for each of 98 candidates, totalling 294 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.6s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

[CV] END ..............................kmeans__n_clusters=99; total time=   1.1s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ..............................kmeans__n_clusters=99; total time=   0.8s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                                       ('log_reg', LogisticRegression())]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [22]:
grid_clf.best_params_

{'kmeans__n_clusters': 96}

In [23]:
grid_clf.score(X_test, y_test)

0.9666666666666667

### 9.1.5 군집을 사용한 준지도 학습

준지도 학습: 군집을 사용하는 또 다른 사례
- 레이블이 없는 데이터가 많고 레이블이 있는 데이터는 적을 때 사용

In [24]:
n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
log_reg.score(X_test, y_test)

0.8777777777777778

전체 데이터셋을 사용했을 때보다 낮은 정확도가 나온 것이 당연함

In [26]:
# 대표이미지: 각 클러스터에서 센트로이드에 가장 가까운 이미지
k = 50
kmeans = KMeans(n_clusters=k)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [27]:
y_representative_digits = np.array([4,8,0,6,8,3,7,7,9,2,
                                    5,5,8,5,2,1,2,9,6,2,
                                    1,6,9,0,8,3,0,7,4,1,
                                    6,5,2,4,1,8,6,3,9,2,
                                    4,2,9,4,7,6,2,3,1,1])

In [28]:
log_reg = LogisticRegression()
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.09333333333333334

In [29]:
percentil_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
  in_cluster = (kmeans.labels_==i)
  cluster_dist = X_cluster_dist[in_cluster]
  cutoff_distance = np.percentile(cluster_dist, percentil_closest)
  above_cutoff = (X_cluster_dist > cutoff_distance)
  X_cluster_dist[in_cluster&above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train[partially_propagated]

In [30]:
log_reg = LogisticRegression()
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9311111111111111

In [31]:
np.mean(y_train_partially_propagated == y_train[partially_propagated])

1.0

능동 학습(active learning)
- 전문가가 학습 알고리즘과 상호작용하여 알고리즘이 요청할 때 특정 샘플의 레이블을 제공

### 9.1.6 DBSCAN

- 밀접된 연속적 지역을 클러스터로 정의
- 모든 클러스터가 충분히 밀집되어 있고 밀집되지 않은 지역과 잘 구분될 때 좋은 성능을 냄

In [32]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons
X,y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [33]:
dbscan.labels_

array([ 0,  0,  1,  1,  0,  2,  8,  3,  4,  2,  3,  4,  5, -1,  1,  4,  6,
        4,  0,  2,  6,  7,  3,  3,  3,  0,  6,  4,  7,  0,  8,  2,  5,  0,
        0,  1,  1,  9,  1,  2,  2,  3,  4, -1,  1,  7,  1,  4,  2,  1,  7,
        4,  8,  9,  1,  3,  4,  1, -1,  4,  2,  2,  8,  3,  5,  4,  0,  4,
       10,  2,  2,  0,  4,  2,  1,  7, -1,  1,  7,  4,  4, -1,  4,  0,  1,
        0,  7,  6,  0,  2,  6,  5,  1, -1, -1,  9,  7,  4, 10,  0,  6,  1,
        0, 11, -1,  0,  4,  2,  2,  8,  7, 10,  7,  8,  0,  2,  8,  7,  3,
        3,  0,  4, -1,  5,  5, 10,  7,  0,  4, -1,  3,  2,  1,  4, 12,  3,
        2,  0,  4,  0,  2,  5,  3,  6,  3,  4,  5,  4, -1,  7, -1,  4, 10,
        4, -1,  0,  3,  6,  3,  5,  3,  7,  2,  2,  2,  4,  4,  4,  1,  3,
        3,  3,  8, 11,  2,  4,  3,  2,  2,  5, -1,  2,  5, -1,  0,  0,  6,
        2,  1,  1,  4,  4,  7,  2,  9,  2,  5,  3,  9,  0,  3,  3,  5,  3,
        0, 10,  1,  0,  4,  6,  1,  4, 12,  0,  0,  4,  0,  0,  0,  3,  5,
        0,  1,  3, -1,  0

-1인덱스는 이 샘플을 이상치로 판단했다는 의미

In [34]:
# core_sample_indices_: 핵심 샘플 인덱스
# components_: 핵심 샘플 자체
len(dbscan.core_sample_indices_)

789

In [35]:
dbscan.core_sample_indices_

array([  0,   1,   2,   3,   4,   5,   7,   8,   9,  10,  11,  12,  14,
        16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        29,  30,  31,  32,  33,  36,  37,  38,  39,  40,  41,  42,  44,
        45,  46,  47,  48,  49,  51,  53,  54,  55,  56,  57,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  77,  79,  80,  82,  84,  85,  86,  87,  89,  90,  91,
        92,  95,  96,  99, 100, 101, 102, 103, 106, 107, 109, 110, 111,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 125, 126, 127, 128,
       130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 151, 153, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 166, 167, 168, 169, 170, 171, 174, 175, 176, 177,
       178, 181, 182, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194,
       196, 197, 198, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210,
       211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [36]:
dbscan.components_

array([[ 1.84900637, -0.13346801],
       [ 1.34865526, -0.44076501],
       [ 0.21645981,  0.93260292],
       ...,
       [ 1.07360792,  0.1221687 ],
       [ 0.19497308, -0.17042044],
       [ 0.52973554, -0.37737607]])

In [37]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

KNeighborsClassifier(n_neighbors=50)

In [38]:
X_new = np.array([[-0.5,0],[0,0.5],[1,-0.1],[2,1]])
knn.predict(X_new)

array([3, 7, 8, 9])

In [39]:
knn.predict_proba(X_new)

array([[0.  , 0.  , 0.  , 0.8 , 0.02, 0.  , 0.  , 0.16, 0.  , 0.  , 0.  ,
        0.02, 0.  ],
       [0.  , 0.02, 0.1 , 0.  , 0.04, 0.  , 0.  , 0.56, 0.  , 0.  , 0.26,
        0.02, 0.  ],
       [0.06, 0.  , 0.  , 0.  , 0.02, 0.16, 0.28, 0.  , 0.48, 0.  , 0.  ,
        0.  , 0.  ],
       [0.06, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.5 , 0.  ,
        0.  , 0.44]])

In [40]:
y_dist, y_pred_idx = knn.kneighbors(X_new, n_neighbors=1)
y_pred = dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist>0.2] = -1
y_pred.ravel()

array([-1, 10,  8, -1])

### 9.1.7 다른 군집 알고리즘

1. 병합군집
> - 클러스터 계층을 밑바닥부터 위로 쌓아 구성
2. BIRCH
> - 대규모 데이터셋을 위해 고안
3. 평균-이동
> - 각 샘플을 중심으로 원을 그린다음 안에 포함된 모든 샘플의 평균을 구해 원의 중심을 평균점으로 이동
4. 유사도 전파
> - 투표 방식을 사용해 샘플은 자신을 대표할 수 있는 비슷한 샘플에 투표
5. 스펙트럼 군집
> - 샘플 사이의 유사도 행렬을 받아 저차원 임베딩을 만듦

## 9.2 가우시안 혼합

- 샘플이 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델
- 클러스터는 타원형

In [45]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [46]:
gm.weights_

array([0.19870878, 0.20661381, 0.59467741])

실제 이 데이터를 생성하기 위해 사용한 가중치

In [47]:
gm.means_

array([[ 1.7582849 , -0.04610358],
       [-0.73663773,  0.56702131],
       [ 0.51130825,  0.24330543]])

In [48]:
gm.covariances_

array([[[ 0.04711884,  0.05726492],
        [ 0.05726492,  0.08587998]],

       [[ 0.05860292,  0.06564307],
        [ 0.06564307,  0.08804121]],

       [[ 0.17108369, -0.10402437],
        [-0.10402437,  0.29191003]]])

In [49]:
gm.converged_

True

In [50]:
gm.n_iter_

16

In [51]:
gm.predict(X)

array([0, 0, 2, 2, 0, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2,
       1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2,
       0, 2, 2, 2, 1, 0, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 1, 1, 2,
       2, 2, 2, 2, 0, 1, 2, 2, 1, 1, 0, 2, 1, 2, 2, 2, 2, 0, 2, 0, 1, 1,
       2, 2, 0, 1, 2, 0, 2, 0, 1, 2, 1, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 0, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2,
       1, 1, 1, 2, 1, 1, 2, 0, 0, 0, 2, 1, 2, 2, 2, 2, 2, 1, 0, 1, 2, 1,
       0, 2, 1, 1, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 1,
       2, 0, 2, 1, 1, 0, 1, 0, 2, 2, 0, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2,
       2, 2, 2, 1, 0, 2, 1, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 2, 1, 2, 1, 2,
       1, 2, 2, 2, 1, 2, 0, 2, 2, 1, 2, 0, 0, 1, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 1, 0, 2, 2, 2, 2, 2, 1, 2, 2, 0, 1, 2,

In [52]:
gm.predict_proba(X)

array([[9.94672956e-001, 1.60982182e-218, 5.32704441e-003],
       [7.06455747e-001, 2.31878278e-183, 2.93544253e-001],
       [1.88367593e-119, 9.76113196e-011, 1.00000000e+000],
       ...,
       [2.08881218e-015, 1.58235616e-103, 1.00000000e+000],
       [3.53277670e-053, 1.48456341e-050, 1.00000000e+000],
       [5.96225991e-025, 1.73665084e-089, 1.00000000e+000]])

In [53]:
X_new, y_new = gm.sample(6)
X_new

array([[-0.2689676 ,  1.25624245],
       [ 0.88032542,  0.95906037],
       [ 1.1418034 , -0.24933208],
       [ 0.59545373,  0.26454693],
       [ 0.3529631 ,  0.0796688 ],
       [ 1.29531181, -0.20470024]])

In [54]:
y_new

array([1, 2, 2, 2, 2, 2])

In [55]:
# 확률 밀도 함수
gm.score_samples(X)

array([-1.14860326e+00, -1.62738645e+00, -1.55991925e+00, -1.59035456e+00,
        1.21445832e-01, -2.15073619e+00, -1.71974033e+00, -1.07284093e-01,
       -1.65357284e+00, -3.44503265e-01, -3.87717126e-01, -1.65842634e+00,
       -1.69542847e+00, -2.12800780e+00, -1.61792659e+00, -1.41164451e+00,
       -1.80739501e+00, -1.73111858e+00, -6.83629982e-01, -1.78641319e+00,
       -1.76725515e+00, -1.45956322e+00, -3.61813063e-01, -3.16045601e-01,
       -8.18629027e-01, -8.21739703e-01, -1.99181989e+00, -1.62696111e+00,
       -1.60837012e+00, -2.09868085e+00, -1.45624180e+00, -2.06517214e+00,
       -1.88547724e+00, -1.27348142e-01, -7.97017476e-01, -1.88266547e+00,
       -1.74369732e+00, -7.46000444e-01, -1.73446973e+00, -1.84760233e+00,
       -1.29090350e+00, -1.62146601e+00, -1.42420076e+00, -1.33559437e+00,
       -1.57982991e+00, -1.88239961e+00, -1.70887056e+00, -1.45893052e+00,
       -2.16389935e+00, -1.56829351e+00, -1.33426117e+00, -1.41024173e+00,
       -1.59754262e+00, -

### 9.2.1 가우시안 혼합을 사용한 이상치 탐지

- 밀도가 낮은 지역에 있는 모든 샘플을 이상치로 볼 수 있음
- 밀도 임곗값을 정할 필요가 있음

In [56]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)
anomalies = X[densities<density_threshold]

### 9.2.2 클러스터 개수 선택하기

- 클러스터가 타원형이거나 크기가 다를 떄 안정적이지 않기 때문에 이너셔나 실루엣 점수를 사용할 수 없음
- BIC, AIC 같은 이론적 정보 기준을 최소화하는 모델을 찾음

BIC, AIC
- 학습할 파라미터가 많은 모델에게 벌칙을 가하고 데이터에 잘 학습하는 모델에게 보상을 더함

In [57]:
gm.bic(X)

2764.6708289798576

In [58]:
gm.aic(X)

2681.238989237161

### 9.2.3 베이즈 가우시안 혼합 모델

In [59]:
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(X)
np.round(bgm.weights_,2)

/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:274: ConvergenceWarning: Initialization 10 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


array([0.11, 0.14, 0.14, 0.16, 0.15, 0.08, 0.11, 0.12, 0.  , 0.  ])

8개의 클러스터가 필요하다는 것

### 9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘

- PCA: 이상치 탐지 기법
- Fast-MCD: 데이터셋을 정제할 때 사용
- 아이솔레이션 포레스트: 고차원 데이터셋에서 이상치 감지를 위한 효율적인 알고리즘
- LOF: 주어진 샘플 주위의 밀도와 이웃 주위의 밀도를 비교
- one-class SVM: 특이치 탐지